<a href="https://colab.research.google.com/github/hvmarco/PV-feasibility/blob/main/project_marcohernandez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PV feasibility analysis
This is the project for the course <i>Python Foundations for Spatial Analysis (2024)</i> taught by Ujaval Gandhi and: Vigna Purohi at Spatial Thoughts.

## Setup and data download

First we need to install some additional libraries and import them.

In [1]:
#install and import necessary packages
%%capture
# use %%capture to prevent printing the output of the cell
!pip install numpy-financial #to use financial functions previously in numpy
!pip install pvlib[optional]  #toolbox for simulating the performance of photovoltaic energy systems https://pvlib-python.readthedocs.io/en/stable/index.html#

import numpy as np
import numpy_financial as npf
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import pvlib


Since we will be using some files with the data (location, components, etc), we clone the repository in the colab environment to import them.

In [2]:
%%capture
#clone the repository to include the files needed
!git clone https://github.com/hvmarco/PV-feasibility.git


In [3]:
#change the path in colab so we can access the files
DATA_FOLDER = 'PV-feasibility'
%cd $DATA_FOLDER

/content/PV-feasibility


## Define location

In [10]:
import time
for x in range(3):
    print(x)
    time.sleep(2) #like this we can ensure that we only send 30 requests per minute

0
1
2


In [7]:
# PVGIS has a limit of 30 calls/second
url_base = f"https://re.jrc.ec.europa.eu/api/MRcalc?"

lat=45
lon=8

#----- set API call parameters ------------------
pvgis_params = dict(
    lat=lat,
    lon=lon,
    usehorizon=1,
    raddatabase='PVGIS-ERA5',
    horirrad=1,
    avtemp=1,
    outputformat='json',)

params = "&".join([f'{key}={value}' for key, value in pvgis_params.items()])
url_pvcalc = f'{url_base}&{params}'
params


'lat=45&lon=8&usehorizon=1&raddatabase=PVGIS-ERA5&horirrad=1&avtemp=1&outputformat=json'

In [17]:
df_csv['url_cw'] = url_pvcalc + '&aspect=' + df_csv['azimuth_cw'].map(str) + '&lat=' + df_csv['lat'].map(str) + '&lon=' + df_csv['lon'].map(str)
df_csv['url_aw'] = url_pvcalc + '&aspect=' + df_csv['azimuth_aw'].map(str) + '&lat=' + df_csv['lat'].map(str) + '&lon=' + df_csv['lon'].map(str)

NameError: name 'df_csv' is not defined

In [15]:
import json  # To convert a JSON strings from APIS to Python objects
import requests # The requests module allows you to send HTTP requests and parse the responses using Python.
import time

san_francisco = (37.7749, -122.4194)

def get_month_rad(source_coordinates):
    parameters = {
        'lat': source_coordinates[0],
        'lon' : source_coordinates[1],
        'usehorizon' : 1,
        'raddatabase' : 'PVGIS-ERA5',
        'horirrad' : 1,
        'avtemp' : 1,
        'outputformat' :'json'
    }

    response = requests.get(
        'https://re.jrc.ec.europa.eu/api/MRcalc?',
        params=parameters)

    if response.status_code == 200:
        data = response.json()
        #summary = data['features'][0]['properties']['summary']
        #distance = summary['distance']
        return data
    else:
        print('Request failed.')
        return -9999
get_month_rad(san_francisco)


Request failed.


-9999

## PV components

In [14]:
from pvlib import pvsystem
#import the components databases as pandas Dataframes formated so that pvlib can use
moddb = pvsystem.retrieve_sam(path = 'CEC Modules.csv')
invdb = pvsystem.retrieve_sam(path = 'CEC Inverters.csv')

#select the module and inverter we will be using
pv_module = moddb['Trina_Solar_TSM_670DEG21C_20']
inverter = invdb['Sungrow_Power_Supply_Co___Ltd___SG250HX_US__800V_']


/usr/local/lib/python3.10/dist-packages/pvlib/pvsystem.py:2085: UserWarning: Original names contain 29 duplicate(s).
  warnings.warn('Original names contain %d duplicate(s).' % n_duplicates)
/usr/local/lib/python3.10/dist-packages/pvlib/pvsystem.py:2089: UserWarning: Normalized names contain 29 duplicate(s).
  warnings.warn(
